- CNN to autoencoder for dim reduction
- Clustering algorithms
- Regression problem with multiple probabilities, as opposed to a classification problem with binary probabilities
- How to tell if points are spherical or not? Or if they're a bit like high dimensional moons, all overlapping and tangled together?
- Look at adding a [calibration plot](https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_curve.html#sphx-glr-auto-examples-calibration-plot-calibration-curve-py)
- Restructure loss function to fully reward a model that correctly predicts at least one of a contiguous stretch of non255 labels as being the correct label.

```
y_true: .....1111.......2222.....33333.........4444.........
y_pred: ....11..........2222........33333............4444...
reward  0000111110000000111100000011111110000000000000000000
```

#### Low precision model:
Of all items labelled as X, how many are actually X?
```
y_true: .......11111.......
y_pred: .....111111111.....
```

#### Low recall model:
Of all items that are actually X, how many are labelled X?
```
y_true: .......11111.......
y_pred: .........1.........
```

Try smoothing out the labels, so that they're gradually become more certain of gestureX and then gradually less certain of gestureX instead of instantly gestureX and instantly not gestureX

Try using a segmentation NN instead of a different NN


New metric: Of all the contiguous sequences, how many of them are classified correctly?

Or, number of timesteps to mean of each gesture???

#### Things to try
- Create a real-time visualiser for the data
- Create a model that uses xcorr (or just corr?) to predict classes.
    - It takes an input class, and figures out which of the given examples in the input class best fits all other examples in that class
    - Then repeat on all other classes
    - The result is an ensamble of one-vs-rest predictors
- Create a wrapper to do some basic hyperparameter tuning based on window size and some other things
- Recurrent/LSTM: Don't know if this will actually help
- Convolutional NN: Couldn't get it working
- Dropout
- Preprocess so that the accelerations are orientation invariant
- Visualise *every* incorrectly labelled observation. i.e. every example of predicted 255, actually 001 on one figure
- Algorithmic simplification of a NN? Given a large NN, can you create a smaller NN which is mathematically identical to the larger one? or identical to within some small error
- Maybe a decision tree that only looks at ~10 features per branch
- Ensamble of models, one for each finger

- AI Assignement 3: Find a general way of classifying benchmark problems. Maybe fourier analysis


### Use mode, median, mean for resolving votes from regression ensambles
- Regression ensembles give you multiple continuous values which need to be resolved to a single value
- the arithmetic mean and the median is bad because it doesn't work for non-gaussian or multimodal data.
- Try use KDE and then finding the mode(s) of the regression outputs to give a good estimate
- also look into voting theory to figure out how to resolve the votes in an optimal way


#### To separate the 255 class from others:
- FFT before doing anomaly detection?
- Dynamic time warping to try automatically cluster the gestures
- Anomoly detection via SVM
- Dynamic time warping as the distance metric for kmeans


1. ~Try expanding the dataset~
2. ~Try just using xcorr~ or dynamic time warping
3. Try recording some examples and visualising them to see the problems


Problem: Need a loss function that better reflects the reality. Just changing the label offset and window size might be detrimental to actual performance.

Maybe set it up so the model just detects if there's a gesture anywhere in the window, and then there's a post processor that only recognises the rising edge of predictions

Figure out some other gestures

~remove gravity from gestures?~

In [ ]:
# Imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0' #Don't print TF INFO messages

import ipywidgets as widgets
import matplotlib as mpl
import json
from ipywidgets import interact
from pprint import pprint
import yaml
import pickle
import datetime
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split

from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
keras.utils.set_random_seed(42)

In [ ]:
# Function and constant definitions

FINGERS = [
    'left-5-x',
    'left-5-y',
    'left-5-z',
    'left-4-x',
    'left-4-y',
    'left-4-z',
    'left-3-x',
    'left-3-y',
    'left-3-z',
    'left-2-x',
    'left-2-y',
    'left-2-z',
    'left-1-x',
    'left-1-y',
    'left-1-z',
    'right-1-x',
    'right-1-y',
    'right-1-z',
    'right-2-x',
    'right-2-y',
    'right-2-z',
    'right-3-x',
    'right-3-y',
    'right-3-z',
    'right-4-x',
    'right-4-y',
    'right-4-z',
    'right-5-x',
    'right-5-y',
    'right-5-z',
]


def make_batches(X, y, t, window_size=10, window_skip=1):
    assert window_skip == 1, 'window_skip is not supported for values other than 1'
    ends = np.array(range(window_size, len(y)))
    starts = ends - window_size
    batched_X = np.empty((ends.shape[0], window_size, X.shape[1]))
    batched_y = np.empty((ends.shape[0],), dtype='object')
    for i, (start, end) in enumerate(zip(starts, ends)):
        # Don't add the X,y pair if it would go over a time boundary
        if any(np.diff(t[start:end]) > np.timedelta64(5, 's')):
            continue
        batched_X[i] = X[start:end]
        batched_y[i] = y[end]
    batched_X = batched_X[pd.notna(batched_y)]
    batched_y = batched_y[pd.notna(batched_y)]
    return batched_X, batched_y


def gestures_and_indices(y):
    labels = sorted(np.unique(y))
    g2i_dict = {g: i for i, g in enumerate(labels)}
    i2g_dict = {i: g for i, g in enumerate(labels)}

    def g2i(g):
        not_list = type(g) not in [list, np.ndarray]
        if not_list:
            g = [g]
        result = np.array([g2i_dict.get(gi, gi) for gi in g])
        return result[0] if not_list else result

    def i2g(i):
        not_list = type(i) not in [list, np.ndarray]
        if not_list:
            i = [i]
        result = np.array([i2g_dict.get(ii, ii) for ii in i])
        return result[0] if not_list else result

    return g2i, i2g


def one_hot_and_back(y_all):
    return (
        lambda y: tf.one_hot(y, len(np.unique(y_all))),
        lambda onehot: tf.argmax(one_hot, axis=1)
    )


def conf_mat(y_true, y_pred, perc=None, hide_zeros=True, ax=None, cbar=True):
    assert perc in ['cols', 'rows', 'both', None]
#     y_pred = np.argmax(model.predict(X, verbose=0), axis=1)
#     y_true = y
    confusion_mtx = tf.math.confusion_matrix(y_true, y_pred).numpy()

    axis = None
    if perc == 'cols':
        axis = 0
        confusion_mtx = confusion_mtx / confusion_mtx.sum(axis=axis) * 100
    elif perc == 'rows':
        axis = 1
        confusion_mtx = (confusion_mtx.T /
                         confusion_mtx.sum(axis=axis) * 100).T
    elif perc == 'both':
        axis = (0, 1)
        confusion_mtx = confusion_mtx / confusion_mtx.sum() * 100
    elif perc is None:
        axis = None

    zero_mask = np.where(confusion_mtx == 0)
    not_zero_mask = np.where(confusion_mtx != 0)
    confusion_mtx = np.round(confusion_mtx).astype(int)

    to_print = np.empty(confusion_mtx.shape, dtype='object')
    to_print[zero_mask] = ''
    to_print[not_zero_mask] = confusion_mtx[not_zero_mask].astype(str)

    labels = [i2g(i).replace('gesture0', 'g') for i in range(confusion_mtx.shape[0])]
        
    sns.heatmap(
        confusion_mtx,
        annot=to_print,
        fmt='',
        xticklabels=labels,
        yticklabels=labels,
        square=True,
        cbar=cbar,
        vmin=confusion_mtx.min(),
#         vmax=confusion_mtx[:-1, :-1].max(),# if perc == None else confusion_mtx.max(),
        ax=ax
    )
    if ax is None:
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
    else:
        ax.set_xlabel('Predicted Label')
        ax.set_ylabel('True Label')
    return confusion_mtx


def plot_timeseries(X, y, t=None, per='dimension', axs=None, draw_text=True):
    # Make sure the given dataset is correctly formatted
    assert X.shape[0] == y.shape[
        0], f'There must be one y value for each X value, but got {X.shape[0]} y values and {y.shape[0]} X values'
    assert X.shape[1] == len(
        FINGERS), f'{X.shape[1]=} doesn\'t equal the number of finger labels ({len(FINGERS)})'
    assert not np.isnan(
        X).any(), f'Input dataset has {np.isnan(X).sum()} NaN values. Should have 0'

    # If we've got many many points, only show an abridged version of the plot
    abridged = X.shape[0] > 4000 or not draw_text
    # Only create new axs if we're not plotting on existing axs
    if axs is None:
        if per == 'dimension':
            nrows, ncols = (3, 1)
        elif per == 'finger':
            nrows, ncols = (5, 2)
        _fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(13, 8))
        if len(axs.shape) > 1:
            axs = axs.T.flatten()
    else:
        assert axs.shape in [
            (3,), (10,)], f'Given axs shape is {ax.shape}, but must only be (3,) or (10,))'
        per = 'dimension' if axs.shape == (3,) else 'finger'

    ymin = float('inf')
    ymax = float('-inf')

    max_std = X.std(axis=0).max()
    for d in range(X.shape[1]):
        if per == 'dimension':
            ax_idx = d % 3
        elif per == 'finger':
            ax_idx = d // 3

        ax = axs[ax_idx]
        data_to_plot = X[:, d]
        ax.plot(
            data_to_plot,
            alpha=np.clip(data_to_plot.std() / max_std, 0.05, 1.0),
            label=FINGERS[d],
            c=None if per == 'dimension' else (
                'tab:red', 'tab:green', 'tab:blue')[d % 3]
        )

        # Set the title of each plot
        if per == 'dimension':
            ax.set_title(f'{FINGERS[d][-1]}')
        elif per == 'finger':
            ax.set_title(f'{FINGERS[d][:-2]}')

        ymax = max(ymax, X[:, d].max())
        ymin = min(ymin, X[:, d].min())

    # Plot the ticks and legend for each axis
    NUM_LABELS = 40 if per == 'dimension' else 40
    TICKS_PER_LABEL = max(1, X.shape[0] // NUM_LABELS)
    for i, ax in enumerate(axs):
        if abridged:
            ax.set_xticks([])
            ax.set_xticklabels([])
        else:
            ax.set_xticks(range(0, X.shape[0], TICKS_PER_LABEL))
            if (per == 'dimension' and i != len(axs)-1) or (per == 'finger' and i % 5 != 4):
                ax.set_xticklabels([])
            elif t is not None:
                ax.set_xticklabels(t[::TICKS_PER_LABEL], rotation=90)
        if per == 'dimension' and draw_text:
            handles, labels = ax.get_legend_handles_labels()
            ax.legend(
                handles,
                labels,
                loc='center left',
                bbox_to_anchor=(1., 0.5)
            )
    # Plot the labels for each timestep and axis
    for dim_idx, ax in enumerate(axs):
        backtrack = 0
        for time in range(X.shape[0]):
#             if abridged:
#                 continue
            if y[time] not in ['gesture0255', 'g255'] and time != X.shape[0]-1:
                backtrack += 1
                continue
            elif y[time] in ['gesture0255', 'g255'] and backtrack == 0:
                continue
            else:
                ax.fill_betweenx(
                    y=[ymin * 0.9, ymax * 1.1],
                    x1=[time - backtrack - .5, time - backtrack - .5],
                    x2=[time - 0.5, time - 0.5],
                    color='grey',
                    alpha=0.1
                )

                txt = y[time - backtrack].replace('gesture0', 'g')
                ax.text(
                    time - backtrack / 2 - .5,
                    (ymax - ymin)/2 + ymin,
                    txt,
                    va='baseline',
                    ha='center',
                    rotation=90
                )
                backtrack = 0
        ax.set_ylim((ymin * 0.9, ymax * 1.1))

    plt.tight_layout()
    return axs


def plot_means(Xs, per='finger'):
    assert Xs.shape[-1] == len(
        FINGERS), f"Xs is of shape {Xs.shape}, not (None, None, {len(FINGERS)})"
    assert len(
        Xs.shape) == 3, f"Xs should have 3 dimensions, not {len(Xs.shape)}"
    X_mean = Xs.mean(axis=0)
    X_std = Xs.std(axis=0)

    blank_labels = np.array(['g255'] * X_mean.shape[0])

    axs = plot_timeseries(
        X_mean,
        blank_labels,
        per=per
    )

    ymin = float('inf')
    ymax = float('-inf')
    max_std = X_mean.std(axis=0).max()

    for d in range(X_mean.shape[1]):
        if per == 'dimension':
            ax_idx = d % 3
        elif per == 'finger':
            ax_idx = d // 3

        ax = axs[ax_idx]

        high = X_mean[:, d] + X_std[:, d]
        low = X_mean[:, d] - X_std[:, d]
        ymin = min(ymin, min(low))
        ymax = max(ymax, max(high))

        kwargs = {} if per == 'dimension' else {
            'color': ('tab:red', 'tab:green', 'tab:blue')[d % 3]}
        ax.fill_between(
            range(len(X_mean[:, d])),
            low,
            high,
            alpha=np.clip(X_mean[:, d].std() / (4*max_std), 0.05, 1.0),
            **kwargs
        )

    for ax in axs:
        ax.set_ylim((ymin * 0.9, ymax * 1.1))

    plt.tight_layout()
    return axs


def plot_mean_gesture(gesture, window_size=15, per='finger'):
    y_orig = df['gesture'].to_numpy()
    X_orig = df.drop(['datetime', 'gesture'], axis=1).to_numpy()
    t_orig = df['datetime'].to_numpy()
    # Get a series which is y_orig, but shifted backwards by one
    y_offset = np.concatenate((['gesture0255'], y_orig[:-1]))
    # Get all the indices where the gesture goes [..., !=gesture, ==gesture, ...]
    indices = np.nonzero(y_orig == gesture)[0]
    # Filter out those indices too close to the starts/finishes for it to be viable
    indices = indices[(indices > window_size) & (
        indices + window_size + 1 < X_orig.shape[0])]

    Xs = np.empty((
        len(indices),
        window_size * 2 + 1,
        X_orig.shape[-1]
    ))

    for i, idx in enumerate(indices):
        window_start = idx - window_size
        window_finsh = idx + window_size + 1
        Xs[i] = X_orig[window_start: window_finsh]

    return plot_means(Xs, per=per)


def parse_csvs(root='../gesture_data/train/'):
    dfs = []
    for path in os.listdir(root):
        dfs.append(pd.read_csv(
            root + path,
            names=['datetime', 'gesture'] + FINGERS,
            parse_dates=['datetime']
        ))
    df = pd.concat(dfs)
#     df.datetime = df.datetime.apply(pd.Timestamp)
    return df


class PerClassCallback(keras.callbacks.Callback):
    """A basic wrapper to calculate the sklearn `classification_report` at 
    the end of each epoch. Made trickier because `validation_data` isn't
    directly available from Keras."""
    def __init__(self, val_data):
        super().__init__()
        self.validation_data = val_data
        
    def on_train_begin(self, logs={}):
        self.model.reports = []

    def on_epoch_end(self, batch, logs={}):
        # Only do expensive logging every ~20 epochs
        if len(self.model.reports) > 10 and np.random.random() > 0.05:
            self.model.reports.append(self.model.reports[-1])
            return
        y_pred = np.argmax(np.asarray(
            self.model.predict(self.validation_data[0], verbose=0)
        ), axis=1)
        y_true = self.validation_data[1]
        self.model.reports.append(classification_report(
            y_true,
            y_pred,
            target_names=i2g(np.unique(y_true)),
            output_dict=True,
            zero_division=0,
        ))
        return


def compile_and_fit(X_train, y_train, X_valid, y_valid, config, verbose=1):
    normalizer = layers.Normalization(axis=-1)
    normalizer.adapt(X_train)
    
    dense_layers = []
    
    for layer_number, num_units in config.get("n_hidden_units").items():
        dense_layers.append(layers.Dense(
            units=num_units,
            activation='relu',
        ))
        dense_layers.append(layers.Dropout(config['dropout_frac']))

    def init_biases(shape, dtype=None):
        assert shape == [len(config['class_weight'])
                         ], f"Shape {shape} isn't ({len(config['class_weight'])},)"
        inv_freqs = np.array([1/v for v in config['class_weight'].values()])
        return np.log(inv_freqs)

    model = tf.keras.Sequential([
        layers.Input(shape=X_train.shape[1:]),
        normalizer,
        layers.Flatten(),
        *dense_layers,
        layers.Dense(
            len(np.unique(y)),
            activation=config.get("activation"),
            bias_initializer=init_biases,
        ),
    ])
    
    # Define an object that calculates per-class metrics
    per_class_callback = PerClassCallback((X_valid, y_valid))

    # Instantiate and compile the model
    model.compile(
        optimizer=config['optimiser'],
        loss=config['loss_fn'],
        weighted_metrics=[
            keras.metrics.SparseCategoricalAccuracy(name='sca'),
            keras.metrics.SparseCategoricalCrossentropy(name='scce')
        ],
    )
    # Fit the model, using the early stopping callback
    history = model.fit(
        X_train,
        y_train,
        verbose=verbose,
        batch_size=config['batch_size'],
        epochs=config['epochs'],
        validation_data=(X_valid, y_valid),
#         class_weight=config['class_weight'],
        callbacks=[
            per_class_callback,
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=25,
                mode='min',
                restore_best_weights=True,
                verbose=0,
            ),
        ],
    )
    return history, model


def plot_losses(history, show_figs, d, results, trimmed_config):
    _fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.plot(
        history.history['scce']
    )
    ax.plot(
        history.history['val_scce']
    )
    plt.title(f'Sparse Categorical Cross Entropy\n{trimmed_config}')

    ylim = ax.get_ylim()
    ax.set_ylim((0, ylim[1]))
    ax.set_ylabel('SCCE')
    ax.set_xlabel('epoch')
    ax.legend(['train', 'val'], loc='best')
    plt.tight_layout()
    plt.savefig(f'{d}/metrics.pdf')
    if show_figs:
        plt.show()
    else:
        plt.close()


def plot_confusion_matrices(
    y_valid, y_pred_valid, y_train, y_pred_train, results, trimmed_config, 
    show_figs, d, cbar=False, perc='both'
):
    fig, axs = plt.subplots(1, 2, figsize=(16, 9))
    _ = conf_mat(y_valid, y_pred_valid, ax=axs[0], perc=perc, cbar=cbar)
    axs[0].set_title(
        f'Validation set (support={len(y_valid)}, $F_1$={results["valid_f1"]})\nscce={results["val_scce"]}, sca={results["val_sca"]}'
    )

    _ = conf_mat(y_train, y_pred_train, ax=axs[1], perc=perc, cbar=cbar)
    axs[1].set_title(
        f'Training set (support={len(y_train)}, $F_1$={results["train_f1"]})\nscce={results["scce"]}, sca={results["sca"]}'
    )

    plt.suptitle(
        f'Validation and Training Confusion Matrices\n{trimmed_config}')

    plt.tight_layout()
    plt.savefig(f'{d}/confusion_matrices.pdf')
    if show_figs:
        plt.show()
    else:
        plt.close()


def plot_metrics(model, d, show_figs):
    # Plot the Precisions, recalls, and F1s for all classes
    shape = (len(model.reports), len(model.reports[0].keys()) - 3)
    precisions = np.zeros(shape)
    recalls = np.zeros(shape)
    f1s = np.zeros(shape)
    for i, report in enumerate(model.reports):
        filtered = {k:v for k, v in report.items() if k.startswith('gesture')}
        precisions[i] = np.array([v['precision'] for k, v in filtered.items()])
        recalls[i]    = np.array([v['recall']    for k, v in filtered.items()])
        f1s[i]   = np.array([v['f1-score']  for k, v in filtered.items()])

    labels = list({k:v for k, v in model.reports[0].items() if k.startswith('gesture')}.keys())
    _fig, axs = plt.subplots(3, 1, figsize=(6, 8))

    val_metrics = [
        (precisions, "Precision"),
        (recalls, "Recall"),
        (f1s, "$F_1$ Score"),
    ]

    for ax, (vals, metric) in zip(axs, val_metrics):
        values = vals[:, np.nonzero(np.array(labels) != 'gesture0255')[0]]
        mean = np.mean(values, axis=1)
        std = np.std(values, axis=1)
        ax.fill_between(
            x=range(len(mean)),
            y1=mean - std,
            y2=mean + std,
            color='tab:orange',
            alpha=0.1,
    #         lw=2
        )
        ax.plot(
            vals[:, np.nonzero(np.array(labels) != 'gesture0255')[0]],
            alpha=0.1,
        )
        ax.plot(
            mean,
            c='tab:orange',
            lw=2
        )
        ax.plot(
            vals[:, np.nonzero(np.array(labels) == 'gesture0255')[0]],
            label='g255',
            c='tab:blue',
            lw=2
        )
        ax.set_ylim((0, 1))
        ax.set_xlabel('Epochs')
        ax.set_ylabel(f'{metric.title()}')
        ax.set_title(f'{metric} for all gesture classes')

    axs[2].legend([
            mpl.lines.Line2D([0], [0], color='tab:blue', lw=4),
            mpl.lines.Line2D([0], [0], color='tab:orange', lw=4),
        ], 
        ['gesture0255', 'mean$\pm$std.dev. for all other gestures'],
        bbox_to_anchor=(0.6, -0.25)
    )
    plt.tight_layout()
    plt.savefig(f'{d}/precision_recall_f1.pdf')
    if show_figs:
        plt.show()
    else:
        plt.close()


def eval_and_save(
    model, X_train, y_train, X_valid, y_valid, config, history, 
    show_figs=False, cbar=True, make_plots=True, perc='both'
):
    print("Saving model")
    d = f'./models/{str(datetime.datetime.now()).replace(" ", "T")}'
    model.save(d)
    with open(f'{d}/config.yaml', 'w') as file:
        config['class_weight'] = {int(k): v for k, v in config['class_weight'].items()}
        config['gestures'] = [int(i) for i in config['gestures']]
        config['i2g'] = {int(i): str(i2g(i)) for i in config['gestures']}
        config['g2i'] = {v: k for k, v in config['i2g'].items()}
        yaml.dump(config, file)

    keys = ['epochs', 'label_expansion', 'n_hidden_units', 'window_size']
    trimmed_config = {k: v for k, v in config.items() if k in keys}
    results = {k: v[-1] for k, v in history.history.items()}
    
    # Collect together the precision/recall/F1 reports and combine them into results
    filtered = {k:v for k, v in model.reports[-1].items() if k.startswith('gesture')}
    merged = [[{f'{k}.{ki}': vi} for ki, vi in v.items()] for k, v in filtered.items()]
    for item in sum(merged, []):
        results.update(item)

    if make_plots:
        print("Making predictions")
        y_pred_valid = np.argmax(model.predict(X_valid, verbose=0), axis=1)
        y_pred_train = np.argmax(model.predict(X_train, verbose=0), axis=1)
        results.update({
            "valid_f1": float(f1_score(y_valid, y_pred_valid, average='weighted')), 
            "train_f1": float(f1_score(y_train, y_pred_train, average='weighted')),
        })
        print("Making plots")
        plot_losses(history, show_figs, d, results, trimmed_config)
        plot_confusion_matrices(
            y_valid, y_pred_valid, y_train, y_pred_train, results, trimmed_config, 
            show_figs, d, cbar, perc=perc
        )
        plot_metrics(model, d, show_figs)


    if os.path.exists('./models/results.jsonlines'):
        old = pd.read_json('./models/results.jsonlines', lines=True)
    else:
        old = pd.DataFrame()
    new = pd.json_normalize(results | config)
    pd.concat((old, new), ignore_index=True).to_json(
        './models/results.jsonlines',
        orient='records',
        lines=True,
    )
    with open(f'{d}/results.yaml', 'w') as file:
        yaml.dump(results, file)


def build_dataset(df, config):
    print(f"Making batches with window size of {config['window_size']}")
    X, y = make_batches(
        df.drop(['datetime', 'gesture'], axis=1).to_numpy(),
        df['gesture'].to_numpy(),
        df['datetime'].to_numpy(),
        window_size=config['window_size'],
        window_skip=config['window_skip'],
    )

    # Offset the label by some amount
    if config.get('label_offset', 0) > 0:
        print(f"Offsetting the labels by {config['label_offset']}")
        padding = np.array(['gesture0255'] * config['label_offset'])
        y = np.concatenate((padding, y[:-config['label_offset']]))
    elif config.get('center_label', False):
        offset = config['window_size'] // 2
        padding = np.array(['gesture0255'] * offset)
        y = np.concatenate((padding, y[:-offset]))

    config['label_before'] = config.get(
        'label_expansion', config.get('label_before', 0))
    config['label_after'] = config.get(
        'label_expansion', config.get('label_after', 0))
    # Extend the labels by a certain amount
    non255_idxs = np.where(y != 'gesture0255')[0]
    for i in range(-config.get('label_before', 0), config.get('label_after', 0)):
        if non255_idxs.max() + i >= y.shape[0] or i == 0:
            continue
        y[non255_idxs + i] = y[non255_idxs]

    if config['label_before'] or config['label_after']:
        print(
            f"Expanding labels to be in deltas of {config['label_before']} before to {config['label_after']} after")

    # print(np.where(y != 'gesture0255')[0][:16])
    if config.get('allowlist', []):
        print(f"Only including gestures in {config['allowlist']}")
        X = X[np.isin(y, config['allowlist'])]
        y = y[np.isin(y, config['allowlist'])]

    if config.get('omit_0255', False):
        print(f"Omitting the 255 gesture")
        X = X[y != 'gesture0255']
        y = y[y != 'gesture0255']

    if config.get('g255_vs_rest', False):
        print(f"Grouping gestures to be gesture0255 vs rest")
        y = np.where(
            y == 'gesture0255',
            'gesture0255',
            'gesture0256'
        )
    config['label_size_over_window_size'] = \
        (config['label_before'] + config['label_after']) / \
        config['window_size']
    # Get functions to convert between gestures and indices
    g2i, i2g = gestures_and_indices(y)

    labels = sorted(np.unique(y))
    g2i_dict = {g: i for i, g in enumerate(labels)}
    i2g_dict = {i: g for i, g in enumerate(labels)}
    print("TODO: Don't store i2g and g2i in saved_models/")
    with open("saved_models/idx_to_gesture.pickle", "wb") as f:
        pickle.dump(i2g_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open("saved_models/gesture_to_idx.pickle", "wb") as f:
        pickle.dump(g2i_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

    y = g2i(y)
    # Get functions to convert between indices and one hot encodings
    i2ohe, ohe2i = one_hot_and_back(y)

    total = len(y)
    n_unique = len(np.unique(y))
    config['gestures'] = list(np.unique(y))

    X_train, X_valid, y_train, y_valid = train_test_split(
        X,
        y,
        test_size=config['test_frac'],
        random_state=42
    )

    class_weight = {
        int(class_): (1/freq) for class_, freq in zip(*np.unique(y_train, return_counts=True))
    }
    class_weight = {int(k): float(v/sum(class_weight.values()))
                    for k, v in class_weight.items()}
    default_class_weights = {
        int(g): float(1.0/n_unique) for g in np.unique(y_train)
    }
    config['class_weight'] = class_weight if config['use_class_weights'] else default_class_weights

    return config, g2i, i2g, i2ohe, ohe2i, X, y, X_train, X_valid, y_train, y_valid

In [ ]:
# Read in the existing config, optionally update it, and then save the result to disk again
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

config.update({
#     'label_offset': 0,
    'center_label': True,
    'label_expansion': 5,
    'window_size': 10,
    'epochs': 500,
    'n_hidden_units': {1: 128, 2: 128},
    'architecture': 'ffnn',
    'dropout_frac': 0.5,
    'use_class_weights': False
})

with open('config.yaml', 'w') as file:
    yaml.dump(config, file)

pprint(config)

TODO: you're not using softmax even though you mention it in the report. Also the biases need to be recalculated because softmax is no longer being used

In [ ]:
df = parse_csvs()

config, g2i, i2g, i2ohe, ohe2i, X, y, X_train, X_valid, y_train, y_valid = \
    build_dataset(df, config)
    
to_print = [
    (i2g(i), c, round(config['class_weight'][i], 6)) 
    for i, c in 
    zip(*np.unique(y, return_counts=True))
]
print('\n'.join([f'{g: <12} {c: >5} {f: >7}' for g, c, f in to_print]))


In [ ]:
# Tensorflow Feed Forward NN
history, model = compile_and_fit(
    X_train, y_train, X_valid, y_valid, config, verbose=1)

print('Percentage g255: ', (y_train == g2i('gesture0255')).sum() / y_train.shape)

In [ ]:
eval_and_save(
    model, X_train, y_train, X_valid, y_valid, config, history, 
    show_figs=True, cbar=False, make_plots=True, perc='both',
)

In [ ]:
model.summary()

In [ ]:
# !rm models/results.jsonlines

In [ ]:
# Train models over a range of hyperparameters
import itertools

config_changes = {
#     'label_expansion': [0, 1, 2, 5, 10, 15, 20],
#     'window_size':     [5, 10, 15, 20, 25, 30, 35, 40],
    'label_expansion': [5, 10],
    'window_size':     [25, 30],
    'n_layers': [1, 2, 3],
    'neurons_per_layer': [64, 128, 256],
}
num_items = np.prod([len(values) for values in config_changes.values()])
pbar = tqdm(total=num_items)

# for key, values in config_changes.items():
for values in itertools.product(*config_changes.values()):
    df = pd.concat((
        parse_csvs(),
    ))
    print("Getting dict")
    update_dict = {k:v for k, v in zip(config_changes.keys(), values)}
    update_dict['n_hidden_units'] = {
        i: update_dict['neurons_per_layer']
        for i
        in range(1, update_dict['n_layers'] + 1)
    }
    new_config = config | update_dict
    pbar.set_description(str(update_dict))
    print("Building Dataset")
    new_config, g2i, i2g, i2ohe, ohe2i, X, y, X_train, X_valid, y_train, y_valid = \
        build_dataset(df, new_config)
    print("Compiling and fitting the model")
    history, model = compile_and_fit(X_train, y_train, X_valid, y_valid, new_config, verbose=0)
    print("Evaluating and saving the model")
    eval_and_save(model, 
                  X_train, y_train, 
                  X_valid, y_valid, 
                  new_config, history, 
                  show_figs=False, make_plots=False
    )
    pbar.update()

In [ ]:
varz = ['label_expansion', 'window_size', 'n_layers', 'neurons_per_layer']
WEIGHTING_g255 = 0.5
WEIGHTING_n255 = 0.9
rename = {
    'label_expansion': 'Label Expansion',
    'window_size': 'Window Size',
    'n_layers': 'Num. Layers',
    'neurons_per_layer': 'Neurons/Layer',
}

value = 'f1-score'
df = pd.read_json('./models/results.jsonlines', lines=True)
other_f1_scores = [c for c in df.columns if not c.startswith('gesture0255') and c.endswith(value)]

fig, axs = plt.subplots(
    len(varz)-1, len(varz)-1,
    figsize=(3*len(varz), 3*len(varz))
)
for x, var1 in enumerate(varz[1:]):
    for y, var2 in enumerate(varz[:-1]):
        if x < y:
            axs[x, y].axis('off')
            continue
        axs[x, y].set_title(f'{rename[var2]} vs {rename[var1]}')
        dedup = df.groupby([var1, var2])[val] \
            .mean() \
            .reset_index()
        g255 = df.groupby([var1, var2])[f'gesture0255.{value}'].mean()
        n255 = df.groupby([var1, var2])[other_f1_scores].mean().mean(axis=1)

        dedup = ((g255*WEIGHTING_g255 + n255*WEIGHTING_n255) /
                 (WEIGHTING_g255+WEIGHTING_n255)).reset_index()
        dedup.columns = list(dedup.columns[:-1]) + [value]
        heatmap = dedup \
            .pivot(index=var1, columns=var2, values=value) \
            .sort_index(ascending=False)
        sns.heatmap(
            heatmap,
            square=True,
            vmax=1,
            annot=True,
            fmt='.2f',
            cbar=False,
            ax=axs[x, y],
        )
        axs[x, y].set_xlabel(rename[var2])
        axs[x, y].set_ylabel(rename[var1])
        
plt.suptitle(f'Mean {value} for different value of:\n{", ".join(rename.values())}')
plt.tight_layout()
plt.savefig('../../report/imgs/hyperpar_tests_weighted.pdf')
plt.show()

In [ ]:
f1_scores = [c for c in df.columns if c.endswith('f1-score')]

dedup = df.groupby(varz)[f1_scores] \
    .mean() \
    .reset_index()
g255 = df.groupby(varz)[f'gesture0255.f1-score'].mean()
n255 = df.groupby(varz)[other_f1_scores].mean().mean(axis=1)

dedup = ((g255*WEIGHTING_g255 + n255*WEIGHTING_n255) /
         (WEIGHTING_g255+WEIGHTING_n255)).reset_index(name='f1-score')

with open('../../report/imgs/best_f1_score.tex', 'w') as f:
    topn = 15
    dedup[varz] = dedup[varz].astype(int)
    tex = (dedup
        .sort_values('f1-score', ascending=False)
        .rename(columns=rename|{
            'f1-score': '$F_1$-score',
        })
        .head(topn)
    )
    tex['$F_1$-score'] = np.round(tex['$F_1$-score'], 3)
    
    f.write(tex.to_latex(
        index=False,
        escape=False,
        caption=f'Top-{topn} Mean $F_1$-scores',
        label='tab:best_f1_score'
    ).replace("{table}", "{table*}"))
tex

In [ ]:
best = dedup.sort_values('f1-score', ascending=False).iloc[0]
best_row = df.loc[
    (df['neurons_per_layer'] == best['neurons_per_layer']) & 
    (df['label_expansion'] == best['label_expansion']) & 
    (df['n_layers'] == best['n_layers']) & 
    (df['window_size'] == best['window_size'])
].iloc[0]


In [ ]:
dyn_content = {
    "best-window-size": str(int(best['window_size'])),
    "best-label-expansion": str(int(best['label_expansion'])),
    "best-num-hidden-layers": str(int(best['n_layers'])),
    "best-nodes-per-layer": str(int(best['neurons_per_layer'])),
    "best-f1-score": str(np.round(best['f1-score'], 4)),
    "best-scce": str(np.round(best_row['val_scce'], 4)),
}
print(dyn_content)
with open('../../report/dynamic_content.yaml', 'w') as f:
    yaml.dump(dyn_content, f)

In [ ]:
import glob
def models_from_config(needle):
    paths = glob.glob('models/*/config.yaml')
    viable_paths = []
    for path in paths:
        with open(path, 'r') as f:
            haystack = yaml.unsafe_load(f)
        all_match = True
        for k1, v1 in needle.items():
            for k2, v2 in haystack.items():
                if k1 != k2:
                    continue
                if v1 != v2:
                    all_match = False
                    break
            if not all_match:
                break
        if all_match:
            viable_paths.append(os.sep.join(path.split(os.sep)[:-1]))
    return viable_paths

paths = models_from_config({
    'label_expansion': 10, 
    'window_size': 25,
    'n_layers': 2,
    'neurons_per_layer': 256,
})
if len(paths) == 1:
    with open(paths[0] + '/config.yaml', 'r') as f:
        config = yaml.unsafe_load(f)
        pprint(config)
    model = keras.models.load_model(paths[0])
    print(paths[0])
else:
    with open(paths[0] + '/config.yaml', 'r') as f:
        config = yaml.unsafe_load(f)
    model = keras.models.load_model(paths[0])
    print(paths)

In [ ]:
# TODO create a widget to explore the precision/recall/f1 graphs for different areas in the heatmap

In [ ]:
# Visualise the predictions
df = pd.concat((
    parse_csvs(),
))
config, g2i, i2g, i2ohe, ohe2i, X, y, X_train, X_valid, y_train, y_valid = \
    build_dataset(df, config)

window_size = config['window_size']
@interact(idx=(2*window_size, len(df) - window_size, 5))
def view_predictions(idx=window_size):
    if idx < 2*window_size or idx > len(df) - window_size:
        print(f"Clamping idx to between {2*window_size} and {len(df) - window_size}")
        idx = min(len(df) - window_size, max(idx, 2*window_size))
    
    y_orig = df['gesture'].to_numpy()
    X_orig = df.drop(['datetime', 'gesture'], axis=1).to_numpy()
    t_orig = df['datetime'].to_numpy()

    s, f = idx-window_size, idx+1#+window_size
    X_window = X[s:f]
    y_window = y[s:f]
    

    shape = model.get_config()['layers'][0]['config']['batch_input_shape']
    assert shape[1] == X_window.shape[1] and shape[2] == X_window.shape[2], \
        f'Shape in config is not the shape of the model'
    proba_preds = model.predict(X_window, verbose=0)
    X_window = X_window[:, 0, :]
    
    mask = np.max(proba_preds, axis=1) < 0.0
    preds = i2g(np.argmax(proba_preds, axis=1))
    preds[mask] = 'gesture0255'
    preds = [g.replace('gesture0', 'g').replace('g255', '') for g in preds]

#     plot_timeseries(
#         X_window,
#         i2g(y_window),
#         preds,
#         per='finger'
#     )
#     plt.suptitle(f'Observation at {idx}')
#     plt.show()
#     # TODO also show prediction probabilities over time

    fig, axs = plt.subplots(2, 1, figsize=(6, 7))
    sns.heatmap(
        X[idx].T,
        xticklabels=[g.replace('gesture0', 'g').replace('g255', '') for g in i2g(y_window)],
        ax=axs[0],
        cbar=None,
        vmax=900,
        vmin=300,
    )
#     axs[0].set_xticklabels(
#         [g.replace('gesture0', 'g').replace('g255', '') for g in i2g(y_window)],
#         rotation=90
#     )
    axs[0].set_title(f'Heatmap of Sensor measurements')
    axs[0].set_xlabel(f'Actual labels')
    axs[0].set_ylabel(f'Sensor')
    cm = plt.get_cmap('tab20')
    NUM_COLORS = len(np.unique(y_orig))

    axs[1].set_prop_cycle(color=([cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]))
    axs[1].plot(
        proba_preds, 
        label=np.unique(y_orig),
    )
    axs[1].set_title(f'Lineplot of model predictions')
    axs[1].set_xticks(range(X[idx].shape[0]))
    axs[1].set_ylim((0, 1))
    axs[1].set_xlabel(f'Predicted Labels')
    axs[1].set_ylabel(f'Softmax of Final Layer')
    axs[1].set_xticklabels(preds[-X[idx].shape[0]:], rotation=90)
    plt.tight_layout()
    plt.show()

In [ ]:
df = parse_csvs()
config, g2i, i2g, i2ohe, ohe2i, X, y, X_train, X_valid, y_train, y_valid = \
    build_dataset(df, config | {'label_expansion': 0})
idx = np.where(y == g2i('gesture0036'))[0][34]
s = idx-105 + 35
f = idx+105 + 35
labels = [str(dt).replace('2022-10-18 ', '') for dt in df.iloc[s:f]['datetime']]
plot_timeseries(X[s:f, -1, :], i2g(y[s:f]), labels, per='finger')
plt.tight_layout()
plt.savefig('../../report/imgs/gesture_over_time.pdf')

# No longer used
- plot the predicted vs actual gestures as a timeline and explore mispredicted items

# TensorFlow CNN
TODO

In [ ]:
cnn_config = {
    'activation': 'softmax',
    'batch_size': 2048,
    'center_label': True,
    'epochs': 500,
    'g255_vs_rest': False,
    'label_expansion': 5,
    'label_offset': 0,
    'loss_fn': 'sparse_categorical_crossentropy',
    'n_hidden_units': {1: 128, 2: 128},
    'omit_0255': False,
    'optimiser': 'adam',
    'test_frac': 0.25,
    'use_class_weights': True,
    'window_size': 20,
    'window_skip': 1,
    'architecture': 'cnn',
}

In [ ]:
import tensorflow as tf

from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

df = parse_csvs()

cnn_config, g2i, i2g, i2ohe, ohe2i, X, y, X_train, X_valid, y_train, y_valid = \
    build_dataset(df, cnn_config)

In [ ]:
print("Compiling and fitting the model")


def compile_and_fit_cnn(X_train, y_train, X_valid, y_valid, config, verbose=0):
    model = models.Sequential()
    model.add(layers.Conv1D(32, (5,), activation='relu', input_shape=X.shape[1:]))
    model.add(layers.MaxPooling1D((2,)))
    model.add(layers.Conv1D(64, (3,), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(len(np.unique(y_train))))
    # Define an object that calculates per-class metrics
    per_class_callback = PerClassCallback((X_valid, y_valid))

    model.compile(
        optimizer=config['optimiser'],
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['sparse_categorical_accuracy', 'sparse_categorical_crossentropy']
    )
    history = model.fit(
        X_train,
        y_train,
        verbose=verbose,
        batch_size=config['batch_size'],
        epochs=config['epochs'],
        validation_data=(X_valid, y_valid),
        class_weight=config['class_weight'],
        callbacks=[
            per_class_callback,
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                mode='min',
                restore_best_weights=True,
                verbose=0,
            )
        ]
    )

    return history, model

history, model = compile_and_fit_cnn(X_train, y_train, X_valid, y_valid, cnn_config, verbose=1)
eval_and_save(model, X_train, y_train, X_valid, y_valid, config, history, show_figs=True)


In [ ]:
# Create a classifier based on correlations

class CorrelationClassifier():
    
    def __init__(self):
        pass
    
    def _get_mean_lag(self,a, b):
        def get_lag(v1, v2):
            x = v1 - np.mean(v1)
            y = v2 - np.mean(v2)
            correlation = scipy.signal.correlate(x, y)
            lags = scipy.signal.correlation_lags(x.size, y.size)
            lag = lags[np.argmax(correlation)]
            return lag, np.max(correlation) / (x.std() * y.std() * (x.shape[0]))

        lags, xcorrs = zip(*[get_lag(ai, bi) for ai, bi in zip(a.T, b.T)])
        weights = [ai.std() * bi.std() for ai, bi in zip(a.T, b.T)]

        total_weight = sum(weights)
        mean_lag = round(sum(w * l / total_weight for w, l in zip(weights, lags)))
        mean_xcorr = sum(w * l / total_weight for w, l in zip(weights, xcorrs))
        return mean_lag, mean_xcorr

    
    def fit(self, X_train, y_train, verbose=1):
        self.X_train = X_train
        self.y_train = y_train
        self.classes_ = sorted(np.unique(y_train))
        self.references = {}
        for cls in self.classes_:
#             if cls == g2i('gesture0255'):
#                 continue
            obs_in_cls = self.X_train[self.y_train == cls]
            if verbose > 0:
                print(f"Fitting class {cls} with {len(obs_in_cls)} observations")
            best_xcorr = 0
            pbar = tqdm.tqdm(total=obs_in_cls.shape[0]*(obs_in_cls.shape[0]-1)//2)
            for i, a in enumerate(obs_in_cls):
                mean_xcorr = 0
                count = 0
                for j, b in enumerate(obs_in_cls):
                    if i >= j:
                        continue
                    pbar.update(1)
                    lag, xcorr = self._get_mean_lag(a, b)
                    mean_xcorr += xcorr
                    count += 1

                if count == 0:
                    best_xcorr = mean_xcorr
                    self.references[cls] = a
                else:
                    mean_xcorr = mean_xcorr / count
                    if mean_xcorr > best_xcorr:
                        best_xcorr = mean_xcorr
                        self.references[cls] = a

    def predict_proba(self, X_valid, verbose=1): 
        corrs_all = []
        for x in tqdm.tqdm(X_valid):
            corrs_all.append([])
            for cls, ref in self.references.items():
                corrs_all[-1].append(self._get_mean_lag(ref, x)[1])
        return np.array(corrs_all)
clf = CorrelationClassifier()
clf.fit(X_train[:1000], y_train[:1000])

In [ ]:
# Calculate and plot metrics per gesture
confusion_mtx = tf.math.confusion_matrix(
    np.argmax(model.predict(X_valid, verbose=0), axis=1), 
    y_valid
).numpy()

tab = pd.DataFrame()
tab['gesture'] = [g for g in i2g(list(range(confusion_mtx.shape[0])))]
tab.index = tab['gesture']
# tab = tab.drop(['gesture'], axis=1)
tab['precision'] = np.diag(confusion_mtx)  / confusion_mtx.sum(axis=0)
tab['recall'] = np.diag(confusion_mtx)  / confusion_mtx.sum(axis=1)
tab['weight'] = pd.Series({i2g(k): v for k, v in config['class_weight'].items()})
tab

## Visualise the mis-predicted gestures

In [ ]:
y_pred = np.argmax(model.predict(X_valid, verbose=0), axis=1)
y_true = y_valid
# incorrect = np.where(y_pred != y_true)[0]

In [ ]:
pred_gestures = i2g(np.unique(y))
true_gestures = i2g(np.unique(y))

@interact(predicted=pred_gestures, true=true_gestures)
def view_incorrect(predicted, true):
    indices = np.where((y_pred == g2i(predicted)) & (y_true == g2i(true)))[0]
    if len(indices) == 0:
        print(f"No gestures where {predicted=} and {true=}")
        return
    
    print(f"Plotting {indices.shape} examples")
    plot_means(X_valid[indices])

    plt.suptitle(f'{len(indices)} gestures where:\npred={predicted}\ntrue={true}')
    plt.tight_layout()
    plt.savefig(f'imgs/gestures_where_pred={predicted}_true={true}.pdf')
    plt.show()


# Relabel part of the dataset

In [ ]:
# Relabel the dataset
gesture = 'gesture0022'
window_size = 10
y_orig = df['gesture'].to_numpy()
ref_idx = np.nonzero(y_orig == gesture)[0][13] - 6

ref_idx = 3598
reference   = df.drop(['datetime', 'gesture'], axis=1).iloc[ref_idx-window_size:ref_idx+window_size].to_numpy()
reference_y = df['gesture'].iloc[ref_idx-window_size:ref_idx+window_size].to_numpy()
reference_t = df['datetime'].iloc[ref_idx-window_size:ref_idx+window_size].to_numpy()

plot_timeseries(
    reference,
    reference_y,
#     reference_t,
    per='finger'
)
plt.suptitle(f"ref index: {ref_idx}")
plt.show()


In [ ]:
# Fix labels
@interact(val=widgets.BoundedIntText(
        value=0,
        min=-window_size,
        max=window_size,
        step=1,
        continuous_update=False
), INDEX=widgets.BoundedIntText(
    value=0,
    min=0,
    max=len(np.nonzero(y_orig == gesture)[0]),
    step=1,
    description='index:'
))
def interact_fix_labels(val=0, INDEX=0):
    y_orig = df['gesture'].to_numpy()
    X_orig = df.drop(['datetime', 'gesture'], axis=1).to_numpy()
    t_orig = df['datetime'].to_numpy()

    # Get a series which is y_orig, but shifted backwards by one
    y_offset = np.concatenate((['gesture0255'], y_orig[:-1]))
    # Get all the indices where the gesture goes [..., !=gesture, ==gesture, ...]
    indices = np.nonzero((y_orig == gesture) & (y_offset != gesture))[0]

    if INDEX >= len(indices):
        print(f"INDEX {INDEX} >= len(indices) {len(indices)}")
        return
    idx = indices[INDEX]
    window_start = idx - window_size
    window_finsh = idx + window_size+1
    X = X_orig[window_start : window_finsh]
    t = t_orig[window_start : window_finsh]
    y_true = y_orig[window_start : window_finsh]
    y_new = y_true.copy()
    y_true = np.array([yi.replace('gesture0255', 'g255') for yi in y_true])

    lag, xcorr = get_mean_lag(reference, X)

    # Remove the old label
    y_new[window_size - 5: window_size + 5] = 'gesture0255'
    # Get indices for the new label
    s = window_size - lag
    f = window_size - lag + 1
    # Set the new label
    y_new[s:f] = gesture

    # Plot the new labels and the old labels
    if True or xcorr < 0.7 or abs(lag) > 5:
        axs = plot_timeseries(
            X,
            y_new,
            y_true,
            per='finger'
        )
        plt.suptitle(f'x-corr: {xcorr:.2f}')

        for ax in axs.flatten():
            ax.axvline(
                window_size - lag, 
                color='red',
                alpha=min(1, xcorr)
            )

#     print(df.loc[df['datetime'].isin(t[s:f]), ['datetime', 'gesture']])
    time_mask = df['datetime'].isin(t)
    df.loc[time_mask, 'gesture'] = y_new
    
    def accept_label(_):
        y_new[window_size - lag] = 'gesture0255'
        y_new[window_size + val] = gesture
        df.loc[time_mask, 'gesture'] = y_new

    button_accept = widgets.Button(
        description='Accept Label',
        button_style='',
    )
    button_accept.on_click(accept_label)
    display(button_accept)

    def remove_label(_):
        y_new[s:f] = 'gesture0255'
        df.loc[time_mask, 'gesture'] = y_new
        print(f'Removed. New label is {y_new}')
    button_remove = widgets.Button(
        description='Remove Label',
        button_style='danger',
    )
    button_remove.on_click(remove_label)
    display(button_remove)
    
    print(f'Time: {t_orig[idx]}, Completed: {INDEX / len(indices) * 100:.0f}%, Gesture: {gesture}, Index: {idx}, \nLag: {lag} \nXcorr: {xcorr}')


In [ ]:
df.to_csv('../gesture_data/relabelled2.csv', index=False)

In [ ]:
# Calculate the optimal lag to align two gestures, and plot the lagged time series
gesture = 'gesture0019'
idxs = np.nonzero((df.gesture == gesture).to_numpy())[0]

i = 40
j = 33

s, f = (idxs[i] - 10, idxs[i] + 10)
a = df.filter(regex='left|right').iloc[s:f].to_numpy()

s, f = (idxs[j] - 10, idxs[j] + 10)
b = df.filter(regex='left|right').iloc[s:f].to_numpy()

def get_lag(v1, v2):
    x = v1 - np.mean(v1)
    y = v2 - np.mean(v2)
    correlation = scipy.signal.correlate(x, y)
    lags = scipy.signal.correlation_lags(x.size, y.size)
    lag = lags[np.argmax(correlation)]
    return lag, np.max(correlation) / (x.std() * y.std() * (x.shape[0]))

def get_mean_lag(a, b):
    lags, xcorrs = zip(*[get_lag(ai, bi) for ai, bi in zip(a.T, b.T)])
    weights = [ai.std() * bi.std() for ai, bi in zip(a.T, b.T)]

    total_weight = sum(weights)
    mean_lag = round(sum(w * l / total_weight for w, l in zip(weights, lags)))
    mean_xcorr = sum(w * l / total_weight for w, l in zip(weights, xcorrs))
    return mean_lag, mean_xcorr

mean_lag, mean_xcorr = get_mean_lag(a, b)

# fig, axs = plt.subplots(len(a.T)//2, 2, figsize=(10, 15))

# for i, (ai, bi) in enumerate(zip(a.T, b.T)):
#     offset = -mean_lag
#     ai = ai[(-offset if offset < 0 else 0):(-offset if offset > 0 else len(ai))]
#     bi = bi[(offset if offset > 0 else 0):(offset if offset < 0 else len(bi))]
#     axs.T.flatten()[i].plot(ai)
#     axs.T.flatten()[i].plot(bi)

# plt.suptitle(f'{mean_lag=}, {mean_xcorr=}')
# plt.tight_layout()

In [ ]:
# Visually confirm that the lag offsetter is working
window_size = 1 + 2 * 10
gesture = 'gesture0019'
idxs = np.nonzero((df.gesture == gesture).to_numpy())[0]
reference_idx = 40

reference = df.filter(regex='left|right').to_numpy()[
    idxs[reference_idx] - window_size // 2: idxs[reference_idx] + window_size // 2
]

def plot_lags(reference, other):
    lag, xcorr = get_mean_lag(reference, other)
    fig, axs = plt.subplots(reference.T.shape[0]//2, 2, figsize=(10, 15))
    for i, (ai, bi) in enumerate(zip(reference.T, other.T)):
        offset = -lag
        axs.T.flatten()[i].plot(
            ai[(-offset if offset < 0 else 0):(-offset if offset > 0 else len(ai))]
        )
        axs.T.flatten()[i].plot(
            bi[(offset if offset > 0 else 0):(offset if offset < 0 else len(bi))]
        )
    for ax in axs.flatten():
        ax.set_xticks([])
        ax.set_yticks([])
    plt.suptitle(f'{lag=}, {xcorr=}')
    return fig, axs

    
# for idx in idxs[:5]:
#     s, f = (idx - window_size // 2, idx + window_size // 2)
#     other = df.filter(regex='left|right').to_numpy()[s:f]
#     plot_lags(reference, other)
#     plt.tight_layout()
#     plt.show()

In [ ]:
gestures = df.reset_index(drop=True).drop_duplicates('gesture', keep='last')['gesture'].sort_values()
gestures

In [ ]:
# Calculate xcorrs between samples from the gestures
pad = 20
xcorrs = np.zeros((len(gestures), len(gestures)))
for i, adx in enumerate(gestures.index):
    for j, bdx in enumerate(gestures.index):
        a = df.filter(regex='left|right').reset_index(drop=True).iloc[adx-pad:adx+pad].to_numpy()
        b = df.filter(regex='left|right').reset_index(drop=True).iloc[bdx-pad:bdx+pad].to_numpy()
        lag, xcorr = get_mean_lag(a, b)
        xcorrs[i, j] = xcorr

In [ ]:
# Plot the cross-correlations heatmap between gestures
sns.heatmap(
    xcorrs,
    annot=True,
    xticklabels=gestures,
    yticklabels=gestures,
    fmt='.1f'
)

plt.title(f'Heatmap of cross correlations for different gestures')
plt.savefig('./imgs/heatmap_of_xcorrs.pdf')

In [ ]:
# Ignore all indices for non-255 gestures that are sequential
pad = 20

for gesture in gestures:
    # Get reference gesture
    reference = np.empty()
    for index in gesture_idxs:
        curr = np.empty()
        # Get lag and xcorr between index and reference
        lag, xcorr = get_mean_lag(reference, curr)
        print(index, lag, xcorr)
        if np.abs(lag) > 0 and xcorr > 0.8:
            df.iloc[index]['gesture']
        

In [ ]:
gesture = 'gesture0012'
index = 3

idxs = np.nonzero((df.gesture == gesture).to_numpy())[0]
s, f = (idxs[index] - 16, idxs[index] + 16)
print(idxs[index])
plot_timeseries(
    df.filter(regex='left|right').iloc[s:f].to_numpy(),
    df['gesture'].iloc[s:f].to_numpy(),
    t=df['datetime'].iloc[s:f].to_numpy(),
    per='finger',
)
plt.show()

In [ ]:
df.gesture.unique()

### Visualise confidence intervals per gesture

In [ ]:
X_orig = df.drop(['datetime', 'gesture'], axis=1).to_numpy()
y_orig = df['gesture'].to_numpy()
indices = np.nonzero(y_orig == 'gesture0255')[0]
indices[0]

In [ ]:
X_orig

In [ ]:

per='finger'
gesture = 'gesture0010'
axs = plot_mean_gesture(gesture, per=per)
# Add some shading to show which 
for ax in axs:
    ylim = ax.get_ylim()
    xlim = ax.get_xlim()
    xrange = range(int((xlim[1] - xlim[0]) // 2 + xlim[0])+1, int(xlim[1]))
    ax.fill_between(
        xrange,
        [ylim[0]] * len(xrange),
        [ylim[1]] * len(xrange),
        color='grey',
        alpha=0.3,
    )
plt.savefig(f'imgs/mean-{gesture}-per-{per}.pdf')


In [ ]:
for gesture in i2g(np.unique(y)):
    per = 'finger'
    plot_mean_gesture(gesture, per=per)
    plt.savefig(f'imgs/mean-{gesture}-per-{per}.pdf')


# TODO: Fix the dataset
- Try train on a small, but very good, dataset

In [ ]:
# # May take a while. Make a pair-plot of the datapoints per sensor
# cols = df.columns[df.columns.str.contains('z')].to_list()
# fig, axs = plt.subplots(len(cols), len(cols), figsize=(len(cols)*2, len(cols)*2))
# # Sort the DF to ensure gesture0255 is drawn first, underneath all others
# df = df.sort_values('gesture', ascending=False)
# for i, x_var in enumerate(cols):
#     print(f'{i} ({x_var}): ', end='')
#     for j, y_var in enumerate(cols):
#         if i > j:
#             print(f'{j}({y_var}) ', end='')
#             axs[i, j].scatter(
#                 df[x_var],
#                 df[y_var],
#                 c=pd.get_dummies(df['gesture']).values.argmax(1),
#                 s=1,
#                 alpha=np.where(df['gesture'] == 'gesture0255', 0.1, 1),
#             )
#         else:
#             axs[i, j].axis('off')
#         if i != len(cols) - 1:
#             axs[i,j].set_xticks([])
#             axs[i,j].set_xticklabels([])
#         else:
#             axs[i,j].set_xlabel(y_var)
            
#         if j != 0:
#             axs[i,j].set_yticks([])
#             axs[i,j].set_yticklabels([])
#         else:
#             axs[i,j].set_ylabel(x_var)
            
#     print()
# print("Resolving layout")
# plt.tight_layout()
# plt.show()

In [ ]:
# Visualise items in the dataset
gesture = 'gesture0000'
if gesture in i2g(np.unique(y)):
    Xs = X[y == g2i(gesture)]
    plot_means(
        Xs
    )
    plt.show()

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Train the outlier detector on a *different* train 
# and validation set to the actuall NN so that the NN's
# validation set doesn't have loads of g0255 and the NN's
# train set has basically none.
X_binary_train, X_binary_valid, y_binary_train, y_binary_valid = train_test_split(
    X, 
    y, 
    test_size=config['test_frac'], 
    random_state=42+1
)


X_binary_train = X_binary_train.reshape((-1, np.prod(X_binary_train.shape[1:])))
y_binary_train = np.where(y_binary_train == g2i('gesture0255'), 1, 0)

X_binary_valid = X_binary_valid.reshape((-1, np.prod(X_binary_train.shape[1:])))
y_binary_valid = np.where(y_binary_valid == g2i('gesture0255'), 1, 0)

clf = RandomForestClassifier(n_estimators=5, class_weight='balanced')
print("Training Random Forest...")
clf = clf.fit(
    X_binary_train,
    y_binary_train,
)
print("Scoring Random Forest...")
clf.score(
    X_binary_valid,
    y_binary_valid,
)
print("Creating confusion matrix...")
y_pred = clf.predict(X_binary_valid)
cm = confusion_matrix(y_binary_valid, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=['not g255', 'g255']
)
disp.plot()
plt.title('Validation Confusion Matrix: g255 vs rest\n(Random Forest Classifier)')
plt.savefig('imgs/conf_mat_255_vs_rest_rfc.pdf')
plt.show()

In [ ]:
with open('./models/255_vs_rest.pickle', 'wb') as f:
    pickle.dump(clf, f)
with open('./models/255_vs_rest.pickle', 'rb') as f:
    clf = pickle.load(f)

Intelligently remove predictions

In [ ]:
# Get a new train-valid split that will be used for the NN
X_train, X_valid, y_train, y_valid = train_test_split(
    X, 
    y, 
    test_size=config['test_frac'], 
    random_state=42
)
# Create reusable masks that only keep observations the RF classifies as `not255`
train_mask = (clf.predict(X_train.reshape((-1, np.prod(X_train.shape[1:]))) != 0))
valid_mask = (clf.predict(X_valid.reshape((-1, np.prod(X_valid.shape[1:]))) != 0))

# Remove all `g255` gestures from the NN's train and valid set
y_train = y_train[train_mask]
X_train = X_train[train_mask]
y_valid = y_valid[valid_mask]
X_valid = X_valid[valid_mask]

# Recalculate the class weights
class_weight = {
    int(class_): (1/freq) for class_, freq in zip(*np.unique(y_train, return_counts=True))
}
class_weight = {k: v/sum(class_weight.values()) for k, v in class_weight.items()}
config['class_weight'] = class_weight if config['use_class_weights'] else None

In [ ]:
to_print = [
    (i2g(i), c, round(config['class_weight'][i], 6)) 
    for i, c in 
    zip(*np.unique(y_train, return_counts=True))
]
print('\n'.join([f'{g: <12} {c: >5} {f: >7}' for g, c, f in to_print]))